In [1]:
from jqdata import *
from jqdata import finance
from jqfactor import get_all_factors 
from jqfactor import get_factor_values
from jqfactor import Factor
from sqlalchemy.sql import func
from sqlalchemy import create_engine
from jqlib.optimizer import *
import pandas as pd   # 0.23.4版本
import os
from datetime import date
# import warnings
# warnings.filterwarnings("ignore")


engine = create_engine("mysql://{}:{}@{}/{}?charset={}"
                       .format('bubble',
                               'bubble',
                               '122.112.170.96',
                               'db_bubble',
                               'utf8'))

# 复权收盘价

In [2]:
# 上证指数收盘价
StartD = '2010-01-01'
EndD = date.today()
# 指标计算和显示用前复权pre，收益计算用后复权post
df = get_price('000001.XSHG', 
          start_date=StartD, end_date=EndD, 
          frequency='daily', fields=['close'], skip_paused=False, fq='pre')
df['code'] = ['000001.SH']*len(df.index)
df = df.reset_index()
df.columns = ['datetime', 'value', 'code']
df.to_sql('factor_index_quote_close', engine, if_exists='append',index=False,chunksize=1000)

# 收盘价预测模型

## 数据预处理

In [2]:

'''
y = 下一日预测目标
x = 上证指数收盘价涨跌幅 + 上证50ETF收盘价 + 北向资金净流入 + 黄金价格 + 债基收益 + 汇率

'''

# 上证指数收盘价涨跌幅, 下一日预测目标
StartD = '2010-01-01'
EndD = date.today()
df = get_price('000001.XSHG', 
          start_date=StartD, end_date=EndD, 
          frequency='daily', fields=['close'], skip_paused=True, fq='pre') 

df['target1'] = df['close'].shift(-1)
df['target2'] = df['close'].shift(-2)
df['target3'] = df['close'].shift(-3)
df['target4'] = df['close'].shift(-4)
df['target5'] = df['close'].shift(-5)
df['close_pct_chg'] = ((df['target1'] - df['close']) / df['close']).shift(1)
df['etf_close'] = get_price('510050.XSHG',
               start_date=StartD, end_date=EndD, frequency='1d', fields=['close'])

# 北向资金净成交数/成交额
df2 = finance.run_query(query(finance.STK_ML_QUOTA.day,
                        finance.STK_ML_QUOTA.buy_amount - finance.STK_ML_QUOTA.sell_amount,
                        finance.STK_ML_QUOTA.buy_volume - finance.STK_ML_QUOTA.sell_volume,
                        ).filter(finance.STK_ML_QUOTA.day >= StartD,
                        finance.STK_ML_QUOTA.link_id==310001).order_by(finance.STK_ML_QUOTA.day.asc()))
df2['day'] = pd.to_datetime(df2['day'])
df2 = df2.set_index('day')
df2.columns = ['net_amt', 'net_vol'] # 单位：笔、亿
df2.index.name = None
df = df.join(df2,how='left')

# 黄金振幅
df2 = finance.run_query(query(finance.FUT_GLOBAL_DAILY.day,
                              finance.FUT_GLOBAL_DAILY.amplitude
                              ).filter(finance.FUT_GLOBAL_DAILY.day>=StartD,
                              finance.FUT_GLOBAL_DAILY.code=='GC'))
df2['day'] = pd.to_datetime(df2['day'])
df2 = df2.set_index('day')
df2.index.name = None
df = df.join(df2,how='left')

df = pd.DataFrame(np.nan_to_num(df), index=df.index, columns=df.columns)
display(df)

,close,target1,target2,target3,target4,target5,close_pct_chg,etf_close,net_amt,net_vol,amplitude
2010-01-04,3243.7600,3282.1800,3254.2200,3192.7800,3196.0000,3212.7500,0.000000,2.052,0.0000,0.0,2.809706
2010-01-05,3282.1800,3254.2200,3192.7800,3196.0000,3212.7500,3273.9700,0.011844,2.078,0.0000,0.0,1.260842
2010-01-06,3254.2200,3192.7800,3196.0000,3212.7500,3273.9700,3172.6600,-0.008519,2.052,0.0000,0.0,2.163225
2010-01-07,3192.7800,3196.0000,3212.7500,3273.9700,3172.6600,3215.5500,-0.018880,2.011,0.0000,0.0,0.950286
2010-01-08,3196.0000,3212.7500,3273.9700,3172.6600,3215.5500,3224.1500,0.001009,2.011,0.0000,0.0,1.808239
2010-01-11,3212.7500,3273.9700,3172.6600,3215.5500,3224.1500,3237.1000,0.005241,2.020,0.0000,0.0,2.107297
2010-01-12,3273.9700,3172.6600,3215.5500,3224.1500,3237.1000,3246.8700,0.019055,2.046,0.0000,0.0,2.952927
2010-01-13,3172.6600,3215.5500,3224.1500,3237.1000,3246.8700,3151.8500,-0.030944,1.956,0.0000,0.0,1.806269
2010-01-14,3215.5500,3224.1500,3237.1000,3246.8700,3151.8500,3158.8600,0.013519,1.974,0.0000,0.0,1.407460
2010-01-15,3224.1500,3237.1000,3246.8700,3151.8500,3158.8600,3128.5900,0.002675,1.979,0.0000,0.0,1.706037


## 机器学习数据切割

In [6]:
from sklearn.model_selection import train_test_split
xlst, ysgn = ['close', 'etf_close', 'net_amt', 'net_vol', 'amplitude'], 'target5'
x, y = df[xlst], df[ysgn]

# METHOD 1
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05, random_state=1)

# METHOD 2
# x_train


## 线性回归

In [7]:
from sklearn.linear_model import LinearRegression

def mx_line(train_x, train_y):
    mx = LinearRegression()
    mx.fit(train_x, train_y)
    print('\nlinreg.intercept_')
    print(mx.intercept_); print(mx.coef_)
    
    return mx

mx = mx_line(x_train.values, y_train.values)
y_pred = mx.predict(x_test.values)
df3 = x_test.copy()
df3['y_test'] = y_test.copy()
df3['y_pred'] = y_pred
df3.sort_index()


linreg.intercept_
54.25308846931148
[0.9657187350145396 22.120406342973457 -0.4301661945318025
 0.0003563484334847544 -2.8037626769298782]


,close,etf_close,net_amt,net_vol,amplitude,y_test,y_pred
2010-01-08,3196.00,2.011,0.0000,0.0,1.808239,3224.15,3180.104430
2010-01-12,3273.97,2.046,0.0000,0.0,2.952927,3246.87,3252.966300
2010-02-11,2985.50,1.853,0.0000,0.0,2.377844,3060.62,2971.728575
2010-03-08,3053.23,1.871,0.0000,0.0,1.718364,2976.94,3039.383897
2010-03-15,2976.94,1.829,0.0000,0.0,0.698920,3074.58,2967.638437
2010-03-22,3074.58,1.891,0.0000,0.0,1.489707,3123.80,3061.085500
2010-05-31,2592.15,1.585,0.0000,0.0,0.692955,2511.73,2590.658870
2010-06-01,2568.28,1.569,0.0000,0.0,1.562886,2513.95,2564.814157
2010-06-21,2586.21,1.591,0.0000,0.0,2.773583,2535.28,2579.221636
2010-08-25,2596.58,1.592,0.0000,0.0,1.013459,2622.88,2594.193230


## 多项式回归

## 预测未来5日收盘价 - LSTM模型

In [ ]:
# import mathimport matplotlib.pyplot as plt
# import keras
# import pandas as pd
# import numpy as np
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import LSTM
# from keras.layers import Dropout
# from keras.layers import * 
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import mean_absolute_error
# from sklearn.model_selection import train_test_split
# from keras.callbacks import EarlyStopping

'''
参考：
https://blog.csdn.net/weixin_26729841/article/details/109069889?utm_medium=distribute.pc_aggpage_search_result.none-task-blog-2~aggregatepage~first_rank_ecpm_v1~rank_v31_ecpm-1-109069889.pc_agg_new_rank&utm_term=用lstm模型预测股票收盘价&spm=1000.2123.3001.4430
https://towardsdatascience.com/time-series-forecasting-predicting-stock-prices-using-an-lstm-model-d0056cd5e055
'''

# from sklearn.preprocessing import MinMaxScaler
# df = df.fillna(method='ffill')
# scaler = MinMaxScaler() # 数据归一化，防止在之后的训练过程中不收敛，feature_range=(-1, 1)
# df['close'] = scaler.fit_transform(df['close'].values.reshape(-1,1))

# 北向因子

## 持仓特征

In [3]:
# factor 月末持仓占流通市值比
finance.run_query(query(finance.STK_HK_HOLD_INFO).filter(   # 数据：持仓占流通市值比
    finance.STK_HK_HOLD_INFO.day=='2022-01-03',
    finance.STK_HK_HOLD_INFO.link_id==310001))  # 310001-沪股通，310002-深股通

,id,day,link_id,link_name,code,name,share_number,share_ratio
0,3758030,2022-01-03,310001,沪股通,600000.XSHG,浦发银行,530513464,1.80
1,3758031,2022-01-03,310001,沪股通,600004.XSHG,白云机场,93996518,4.54
2,3758032,2022-01-03,310001,沪股通,600006.XSHG,东风汽车,21792,0.00
3,3758033,2022-01-03,310001,沪股通,600007.XSHG,中国国贸,10719521,1.06
4,3758034,2022-01-03,310001,沪股通,600008.XSHG,首创股份,149850092,2.04
5,3758035,2022-01-03,310001,沪股通,600009.XSHG,上海机场,186972898,17.09
6,3758036,2022-01-03,310001,沪股通,600010.XSHG,包钢股份,1016656066,3.20
7,3758037,2022-01-03,310001,沪股通,600011.XSHG,华能国际,112660570,1.02
8,3758038,2022-01-03,310001,沪股通,600012.XSHG,皖通高速,35003865,3.00
9,3758039,2022-01-03,310001,沪股通,600015.XSHG,华夏银行,327873168,2.55


In [3]:
# 市场通成交与额度信息表
finance.run_query(query(
    finance.STK_ML_QUOTA).filter(
    finance.STK_ML_QUOTA.day>='2022-01-01', 
    finance.STK_ML_QUOTA.link_id==310001).limit(50))

,id,day,link_id,link_name,currency_id,currency_name,buy_amount,buy_volume,sell_amount,sell_volume,sum_amount,sum_volume,quota,quota_balance,quota_daily,quota_daily_balance
0,6679,2022-01-04,310001,沪股通,110001,人民币,294.3933,1468233.0,303.8004,1498054.0,598.1937,2966287.0,None,None,520.0,512.1693
1,6687,2022-01-05,310001,沪股通,110001,人民币,288.9516,1512384.0,259.7519,1429022.0,548.7035,2941406.0,None,None,520.0,473.8586
2,6697,2022-01-06,310001,沪股通,110001,人民币,252.5915,1295015.0,274.7640,1441370.0,527.3555,2736385.0,None,None,520.0,524.6659
3,6707,2022-01-07,310001,沪股通,110001,人民币,305.3653,1556156.0,234.4542,1292391.0,539.8195,2848547.0,None,None,520.0,431.3549
4,6711,2022-01-10,310001,沪股通,110001,人民币,268.2606,1412521.0,239.1225,1378296.0,507.3831,2790817.0,None,None,520.0,473.2032
5,6719,2022-01-11,310001,沪股通,110001,人民币,256.6014,1296303.0,265.9286,1495602.0,522.5300,2791905.0,None,None,520.0,512.1856
6,6729,2022-01-12,310001,沪股通,110001,人民币,270.4845,1495768.0,222.6520,1320858.0,493.1365,2816626.0,None,None,520.0,452.8233
7,6739,2022-01-13,310001,沪股通,110001,人民币,267.0242,1381218.0,252.4826,1411210.0,519.5068,2792428.0,None,None,520.0,488.4319
8,6749,2022-01-14,310001,沪股通,110001,人民币,251.0091,1311352.0,258.5441,1513248.0,509.5532,2824600.0,None,None,520.0,511.7341
9,6753,2022-01-17,310001,沪股通,110001,人民币,223.0519,1160822.0,219.8205,1262602.0,442.8724,2423424.0,None,None,520.0,499.3380


# 大盘期权数据

In [ ]:
from jqdata import *
opt.run_query(query(opt.OPT_CONTRACT_INFO).filter(opt.OPT_CONTRACT_INFO.code==code).limit(n))

合约标的：
1. 上证50交易型开放式指数证券投资基金（"50ETF"）
2. 华泰柏瑞沪深300交易型开放式指数证券投资基金（证券简称：沪深300ETF，证券代码：510300）